# Домашнее задание №6
## Задание 1
Написать бота который будет проверять доступность сайта

In [ ]:
import telebot
import urllib.request


bot = telebot.TeleBot('token') # импортируем бота
'''Тело бота'''
@bot.message_handler(content_types=['text'])
def get_text_message(message):
	if message.text == "/start":
		bot.send_message(message.from_user.id, "Введи URL сайта")
	elif message.text[:4] == "http":
		try:
			avail = urllib.request.urlopen(message.text).getcode()
			bot.send_message(message.from_user.id, "Сайт доступен!")
		except:
			bot.send_message(message.from_user.id, "Сайт НЕдоступен!")
	else:
		bot.send_message(message.from_user.id, "Хочешь проверить сайт на доступность?\nНапиши /start") 
		print(f'ОТВЕТ ПОЛЬЗОВАТЕЛЯ {message.from_user.first_name} {message.from_user.last_name}: {message.text}')
'''Тело бота'''
bot.polling(none_stop=True, interval=0) # говорим, что бот работает непрерывно

## Задание 2
Написать бота который будет анализировать текст

In [ ]:
import telebot
from re import sub
import os


bot = telebot.TeleBot('token') # импортируем бота
'''Тело бота'''
@bot.message_handler(content_types=['document'])
def get_text_message(message):
	bot.send_message(message.from_user.id, "Анализ текста.\n(Из анализа будут исключены все союзы и предлоги)")
	file_name = message.document.file_name
	file_id = message.document.file_name
	file_id_info = bot.get_file(message.document.file_id)
	downloaded_file = bot.download_file(file_id_info.file_path)
	with open("/Users/pavelzurkin/Desktop/TelBot/" + message.document.file_name, 'wb') as new_file:
            new_file.write(downloaded_file)
	with open("/Users/pavelzurkin/Desktop/TelBot/" + message.document.file_name, 'r', encoding="utf-8") as file:
		text_f = file.read()
	os.remove("/Users/pavelzurkin/Desktop/TelBot/" + message.document.file_name)
	document_analysis(text_f, message)

@bot.message_handler(content_types=['text'])
def get_text_message(message):
	if message.text == "/start":
		bot.send_message(message.from_user.id, "Введи текст, который хочешь проанализировать.\n(Из анализа будут исключены все союзы и предлоги)")
		bot.register_next_step_handler(message, word_analysis)
	else:
		bot.send_message(message.from_user.id, "Для анализа текста напиши /start") 
		print(f'-----\nДо написания /start: {message.text}\n({message.from_user.first_name} {message.from_user.last_name})')

def document_analysis(text, message):
	symbs = '[<|>|{|}|(|)|*|+|=|-|-|—|_|;|:|,|.|!|?|/|#|№|%|$|@|&|1|2|3|4|5|6|7|8|9|0]'
	sz_and_prd = ['и','или','из','за','но','ни','то','хотя','ибо','чтоб','чтобы','зато','без','между','под','в','на','по','вокруг','о','про','до','об','с','для','около','за','от','к','перед','a','как','тоже','если','пока','когда', 'бы']
	long_predlogs = [' потому что ',' как будто ',' то есть ',' так как ',' несмотря на то что ',' благодаря тому что ', ' если бы ', ' как бы ']
	text = text.replace('...', '.') # убрать многоточие для корректного подсчета предложений
	count_sentences = text.count('.') + text.count('!') + text.count('?')
	if text.strip()[-1] != '.' and text.strip()[-1] != '!' and text.strip()[-1] != '?':
		count_sentences += 1
	text = sub(symbs, ' ', text) # очистка от символов и цифр
	text = sub(" +", " ", text) # удаление лишних пробелов между словами
	text = text.lower()
	for predlog in long_predlogs: # очистка от длинных предлогов (состоящих из двух слов)
		text = text.replace(predlog, ' ')
	text = [word for word in text.split() if word not in sz_and_prd] # очистка от союзов и простых предлогов (состоящих из одного слова)
	occur_most_common = 0 # количество вхождений слова
	most_lenght = 0
	word_most_common = ''
	word_most_lenght = ''
	count_unq_word = len(set(text))
	for word in list(set(text)):
		occur = text.count(word) # сколько раз встречается слово
		if occur > occur_most_common: # "!=1" чтобы не считались союзы
			occur_most_common = occur
			word_most_common = word
		if len(word) > most_lenght:
			most_lenght = len(word)
			word_most_lenght = word
	bot.send_message(message.from_user.id, f'Наиболее часто встречающееся: "{word_most_common}" (кол-во {occur_most_common})\nСамое длинное слово: "{word_most_lenght}" (длина {most_lenght})\nКоличество предложений: {count_sentences}\nКоличество уникальных слов: {count_unq_word} (без союзов и предлогов)')

def word_analysis(message):
	text = message.text
	print(f'-----\nКакой текст ввел:\n{text}\n({message.from_user.first_name} {message.from_user.last_name})')
	symbs = '[<|>|{|}|(|)|*|+|=|-|-|—|_|;|:|,|.|!|?|/|#|№|%|$|@|&|1|2|3|4|5|6|7|8|9|0]'
	sz_and_prd = ['и','или','из','за','но','ни','то','хотя','ибо','чтоб','чтобы','зато','без','между','под','в','на','по','вокруг','о','про','до','об','с','для','около','за','от','к','перед','a','как','тоже','если','пока','когда', 'бы']
	long_predlogs = [' потому что ',' как будто ',' то есть ',' так как ',' несмотря на то что ',' благодаря тому что ', ' если бы ', ' как бы ']
	text = text.replace('...', '.') # убрать многоточие для корректного подсчета предложений
	count_sentences = text.count('.') + text.count('!') + text.count('?')
	if text.strip()[-1] != '.' and text.strip()[-1] != '!' and text.strip()[-1] != '?':
		count_sentences += 1
	text = sub(symbs, ' ', text) # очистка от символов и цифр
	text = sub(" +", " ", text) # удаление лишних пробелов между словами
	text = text.lower()
	for predlog in long_predlogs: # очистка от длинных предлогов (состоящих из двух слов)
		text = text.replace(predlog, ' ')
	text = [word for word in text.split() if word not in sz_and_prd] # очистка от союзов и простых предлогов (состоящих из одного слова)
	occur_most_common = 0 # количество вхождений слова
	most_lenght = 0
	word_most_common = ''
	word_most_lenght = ''
	count_unq_word = len(set(text))
	for word in list(set(text)):
		occur = text.count(word) # сколько раз встречается слово
		if occur > occur_most_common: # "!=1" чтобы не считались союзы
			occur_most_common = occur
			word_most_common = word
		if len(word) > most_lenght:
			most_lenght = len(word)
			word_most_lenght = word
	bot.send_message(message.from_user.id, f'Наиболее часто встречающееся: "{word_most_common}" (кол-во {occur_most_common})\nСамое длинное слово: "{word_most_lenght}" (длина {most_lenght})\nКоличество предложений: {count_sentences}\nКоличество уникальных слов: {count_unq_word} (без союзов и предлогов)')
'''Тело бота'''
bot.polling(none_stop=True, interval=0) # говорим, что бот работает непрерывно


## Задание 3
Бот калькулятор

*Примечание: вызывается модуль из файла "Calculator", написанного ранее в домашних заданиях.*

In [ ]:
import telebot
import calculator


trig = False
s = ''
bot = telebot.TeleBot('token') # импортируем бота
'''Тело бота'''
@bot.message_handler(content_types=['text'])
def get_text_message(message):
	global trig
	global s
	if message.text == "/start":
		s = ''
		trig = True
		keyboard = telebot.types.ReplyKeyboardMarkup(True)
		keyboard.row('7', '8', '9', '/')
		keyboard.row('4', '5', '6', '*')
		keyboard.row('1', '2', '3', '-')
		keyboard.row('0', ',', 'Enter', '+')
		bot.send_message(message.from_user.id, "Введи арифметическое выражение, которое требуется посчитать.", reply_markup = keyboard)		
	elif trig == False:
		bot.send_message(message.from_user.id, "Для старта калькулятора отправь\n/start") 
	elif message.text == "AC":
		bot.send_message(message.from_user.id, "Для старта калькулятора отправь\n/start")
		s = ''
		trig = False
	elif message.text == "Enter":
		answer = str(calculator.calculator(s))
		if answer.replace(' ', '').replace('?', '').replace('!', '').isalpha() == False:
			bot.send_message(message.from_user.id, f'{s} = {answer.replace(".", ",")}')
		else:
			bot.send_message(message.from_user.id, f'ОШИБКА: {answer.replace(".", ",")}')
		bot.send_message(message.from_user.id, "Для старта калькулятора отправь\n/start")
		s = ''
		trig = False
	else:	
		s += message.text
'''Тело бота'''
bot.polling(none_stop=True, interval=0) # говорим, что бот работает непрерывно
